In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
#from itertools import combinations
#from itertools import permutations
#from itertools import chain
from datetime import date
from datetime import time
from datetime import datetime

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1813,tt0195714,23000000,53302314,Final Destination,Devon Sawa|Ali Larter|Kerr Smith|Tony Todd|Sea...,James Wong,No accidents. No coincidences. No escapes. You...,After a teenager has a terrifying vision of hi...,98,Horror,New Line Cinema|Zide-Perry Productions|Hard Ei...,3/16/2000,6.2,2000
1286,tt0882977,35000000,42930462,Snitch,Dwayne Johnson|Barry Pepper|Jon Bernthal|Susan...,Ric Roman Waugh,How far would you go to save your son?,Construction company owner John Matthews learn...,112,Thriller|Drama,Front Street Productions|Participant Media|Ima...,2/21/2013,5.7,2013
993,tt1371111,102000000,130482868,Cloud Atlas,Tom Hanks|Halle Berry|Jim Broadbent|Hugo Weavi...,Lilly Wachowski|Lana Wachowski|Tom Tykwer,Everything is Connected,A set of six nested stories spanning time betw...,172,Drama|Science Fiction,Anarchos Productions|X-Filme Creative Pool|Asc...,10/26/2012,6.5,2012
1379,tt0393162,30000000,76669806,Coach Carter,Samuel L. Jackson|Rob Brown|Robert Ri'chard|Ri...,Thomas Carter,It begins on the street. It ends here.,"Based on a true story, in which Richmond High ...",136,Drama,Paramount Pictures|MTV Films|Tollin/Robbins Pr...,1/14/2005,7.3,2005
588,tt0234829,34000000,19693891,Summer Catch,Freddie Prinze Jr.|Jessica Biel|Fred Ward|Matt...,Mike Tollin,Are you game?,A coming-of-age romantic comedy set against th...,108,Drama|Comedy|Romance,Tollin/Robbins Productions|Warner Bros.,8/22/2001,4.8,2001


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [3]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:
profit = data['revenue'] - data['budget'] # добавляем пеменную profit
data['profit'] = data['revenue'] - data['budget'] # добавляем столбец profit

#the time given in the dataset is in string format.
#So we need to change this in datetime format

data['release_date'] = pd.to_datetime(data['release_date'])

# Добавим функцию fun_explode, т.к. часто ее используем в ответах.
# Функция принимает на вход датафрейм и название колонки.
# Далее происходит разделение текста в ячейках в этой колонке по вертикальной черте '|'
# Далее мы растягиваем датафрейм по значениям из полученного списка.
def fun_explode(df, col):
    df[col] = df[col].str.split('|')
    df = df.explode(col)
    return df # возвращаем исправленный датафрейм

In [8]:
df_new = data.copy()
df_new['release_month'] = df_new.release_date.dt.month
df_new.head()



,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,1363528810,6
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,228436354,5
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,185238201,3
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,7.5,2015,1868178225,12
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,7.3,2015,1316249360,4


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [21]:
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +

In [7]:
data_1 = data.copy()
data_1[data_1.budget == data_1.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

In [132]:
test = data.copy()
test.genres = test.genres.str.split('|') # превращаем значение в каждой ячейке в список. 
test = test.explode('genres') # "растягиваем" датафрейм по столбцу genres.
test

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,1363528810
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,228436354
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,185238201
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,7.5,2015,1868178225
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,7.3,2015,1316249360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,2000-07-13,6.6,2000,82299717
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,2000-10-27,6.4,2000,-8444012
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,2000-06-30,4.0,2000,-40865180
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...,Diane Keaton,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,Comedy|Drama,Laurence Mark Productions|Columbia Pictures Co...,2000-02-16,5.2,2000,-3962091


# 2. Какой из фильмов самый длительный (в минутах)?

In [23]:
answers['2'] = '1157. Gods and Generals (tt0279111)'
# +

In [24]:
data_2 = data.copy()
data_2[data_2.runtime == data_2.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [25]:
answers['3'] = '768. Winnie the Pooh (tt1449283)'
# +

In [26]:
data_3 = data.copy()
data_3[data_3.runtime == data_3.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [27]:
answers['4'] = '110'
# +

In [28]:
data_4 = data.copy()
data_4.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [29]:
answers['5'] = '107'
# +

In [8]:
data_5 = data.copy()
data_5.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [31]:
answers['6'] = '239. Avatar (tt0499549)'
# +

In [32]:
# находим строку, соответствующую наибольшему значению profit в нашем файле
data_6 = data.copy()
data_6[profit == profit.max()].original_title

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [33]:
answers['7'] = '1245. The Lone Ranger (tt1210819)'
# +

In [34]:
# находим строку, соответствующую наименьшему значению profit в нашем файле
data_7 = data.copy()
data_7[profit == profit.min()].original_title	 

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [35]:
answers['8'] = '1478'
# +

In [36]:
data_8 = data.copy()
data_8[data_8.revenue > data_8.budget].imdb_id.value_counts().sum()

1478

Вариант 2. 

In [37]:
count = 0
for i in range(0,len(data)): # проходим по строкам нашего файла от нулевой до последней
    if data.revenue[i] > data.budget[i]: # ставим условие: объем сборов больше объема бюджета
               count += 1 # если условие выполнено увеличивам счетчик на 1
count # итоговая сумма счетчик ана экран - это и есть ответ

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [38]:
answers['9'] = '599. The Dark Knight (tt0468569)'
# +

In [39]:
revenue_2008 = [] #создаем список для записи всех сборов 2008 г.
# проходим по строкам нашего файла и, если год = 2008 г., то добавляеем значение кассовых сборов в список выше
for i in range(0,len(data)):  
    if data.release_year[i] == 2008:
        revenue_2008.append(data.revenue[i]) 
# находим фильм, соответствующий максимальному сбору за 2008 г. 
data[data.revenue == max(revenue_2008)].original_title

599    The Dark Knight
Name: original_title, dtype: object

Вариант 2.

In [40]:
data_9 = data.copy() # копируем датафрейм в переменную для этого ответа
data_9 = data_9[data_9.release_year == 2008] # фильтр = 2008 год
data_9[data_9.revenue == data_9.revenue.max()].original_title # из них тот, у которого максимальные сборы

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [41]:
answers['10'] = '1245. The Lone Ranger (tt1210819)'
# +

In [42]:
profit_12_14 = [] # список для записи профита фильмов с 2012 по 2014 годы
# пройдемся по строкам файла и запишем profit для каждого фильма с 2012 по 2014 годы в заготовленный список
for i in range(0,len(data)):
    if 2012 <= data.release_year[i] <= 2014:
        profit_i = data['revenue'][i] - data['budget'][i]
        profit_12_14.append(profit_i)
data[profit == min(profit_12_14)].original_title # найдем минимальный profit за указанный период

1245    The Lone Ranger
Name: original_title, dtype: object

Вариант 2

In [43]:
data_10 = data.copy() 
data_10 = data_10[(data_10.release_year >= 2012) & (data_10.release_year <= 2014)] # фильтр по диапазону дат
data_10[data_10.profit == data_10.profit.min()].original_title # по таблице с необходимыми годами выбираем тот, что самый убыточный

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [44]:
answers['11'] = 'Drama'
#+

In [127]:
df_11 = data.copy()
df_genres = df_11.genres.str.split('|') # создаем списки из жанров, которые хранятся в поле genres, для каждой строки
df_genres_lst = df_genres.sum() # склеиваем все в один список
Counter(df_genres_lst).most_common(1) #считаем количество появлений для каждого слова в списке, выводим максимальный

[('Drama', 782)]

In [5]:
pd.Series(data.genres.str.split('|').sum()).value_counts().index[0]

'Drama'

In [20]:
def stringtolist (string):
    result = string.str.split('|')
    return result
data['genrelist'] = stringtolist(data['genres'])
genrelist = data['genrelist'].sum()
Counter(genrelist).most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [46]:
answers['12'] = 'Drama'
#+

In [9]:
# выбиhаем те фильмы, где прибыль больше 0, т.е. фильм прибыльный
# считаем количество встреч того или иного жанра способом из прошлого шага
data_profit = data[profit > 0]
pd.Series(data_profit.genres.str.split('|').sum()).value_counts().index[0]

'Drama'

In [36]:
profit_genres = data[data['profit']>0] # получили отфильтрованный датафрейм
# далее превратили строки с жанрами в списки, растянули с помощью explode() и посчитали значения с помощью value_counts()
profit_genres.genres.str.split('|').explode().value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

In [17]:
Counter(data[profit > 0].genres.str.split('|').sum()).most_common(1)

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [48]:
answers['13'] = 'Peter Jackson'
#+

In [49]:
data_13 = data.copy()
#вызываем функцию из предобработки:
data_13 = fun_explode(df = data_13, col = 'director')
# Группируем таблицу по режиссерам, считая для каждого из них суммарный доход от фильмов. 
data_13.groupby(['director'])['revenue'].sum().sort_values(ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [50]:
answers['14'] = 'Robert Rodriguez'
#+

In [138]:
data_14 = data.copy()
data_14 = fun_explode(data_14, 'director')
# делаем срез: те строки, где содержится в тексет слово Action
data_14 = data_14[data_14.genres.str.contains('Action', na=False)]
# сколько раз встретился тот или иной режиссер, берем первого с индексом [0]
data_14.director.value_counts().index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [52]:
answers['15'] = 'Chris Hemsworth'
#+

In [150]:
data_15 = data.copy()

# Сначала выберем год:
data_15 = data_15[data_15.release_year == 2012]
# Теперь среди этих фильмов выберем с наибольшим кассовым сбором и выведем на экран список актеров
print(*data_15[data_15.revenue == data_15.revenue.max()].cast)

Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chris Hemsworth|Scarlett Johansson


In [155]:
data_15 = data.copy()
data_2012 = data_15[data_15.release_year == 2012]
print(*data_2012[data_2012.revenue == data_2012.revenue.max()].cast)

Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chris Hemsworth|Scarlett Johansson


In [156]:
data_2012 = data[data.release_year == 2012]
data_2012.cast = data_2012.cast.apply(lambda x: x.split('|'))
data_2012 = data_2012.explode('cast')
display(data_2012.groupby('cast')['revenue'].sum().sort_values(ascending=False).head(3))

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


cast
Chris Hemsworth    2027450773
Denis Leary        1629460639
Anne Hathaway      1522851057
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [54]:
answers['16'] = 'Matt Damon'
#+

In [33]:
data_16 = data.copy()
data_16 = data_16[data.budget > data.budget.mean()] #высокобюджетные фильмы, т.е. бюджет выше среднего
data_16 = fun_explode(data_16, 'cast') # растягиваем
data_16.cast.value_counts() #считаем количество встреч каждого имени, выбираем первого с индексом [0]

Matt Damon                18
Adam Sandler              17
Angelina Jolie            16
Tom Cruise                15
Samuel L. Jackson         15
                          ..
Fred Williamson            1
Willow Smith               1
Cedric the Entertainer     1
Moon Bloodgood             1
Sela Ward                  1
Name: cast, Length: 1505, dtype: int64

In [44]:
movie_data = data.copy()
movie_data = movie_data[movie_data.budget > movie_data.budget.mean()]
actor = movie_data['cast'].str.cat(sep='|') #превращаем список актеров в одну строку
actor = pd.Series(actor.split('|')) #создаем список, разделив строку знаком '|'
actor = actor.value_counts(ascending=False) #считаем количество актеров  в строке
display(actor.head(1)) #выводим результат
#movie_data.groupby(['cast'])[['revenue']].max().sort_values(['revenue'],ascending=False).head(24)

Matt Damon    18
dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [56]:
answers['17'] = 'Action'
#+

In [57]:
data_17 = data.copy()
data_17 = data_17[data_17.cast.str.contains('Nicolas Cage')] # выбрали те строки, где есть Николас Кейдж
data_17 = fun_explode(data_17, 'genres')
# посчитали количество встреч того или иного жанра и берем первую строку - самое большое кол-во
data_17.genres.value_counts().index[0] 

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [58]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
#+

In [59]:
# фильтр на фильмы от Paramount Pictures
data_paramount = data[data.production_companies.str.contains('Paramount Pictures')] 
# выбираем из них с минимальным profit
data_paramount_min = data_paramount[data_paramount.profit == data_paramount.profit.min()] 
data_paramount_min['original_title'] # название на экран

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [60]:
answers['19'] = '2015'
#+

In [31]:
# группируем по году, считаем сумму сбора для каждого года, сортируем по убыванию, выводим первый элемент с индексом "0"
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)
#data.groupby(by='release_year')['revenue'].sum()

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [62]:
answers['20'] = '2014'
#+

In [63]:
# делаем фильтр, только те строки, где есть Warner Bros, далее также, как и на предыдущем шаге
data_WB = data[data.production_companies.str.contains('Warner Bros')] 
data_WB.groupby(['release_year'])['revenue'].sum().sort_values(ascending = False).index[0] 

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [64]:
answers['21'] = 'Сентябрь'
#+

In [65]:
data_21 = data.copy()
# т.к. мы поменяли формат столбца 'release_date' и он стал формата "pandas._libs.tslibs.timestamps.Timestamp"
# с помощью метода 'month' и добавлением 'dt' считаем количество появлений того или иного месяца
data_21.release_date.dt.month.value_counts().index[0] 

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [66]:
answers['22'] = '450'
#+

In [67]:
data_22 = data.copy()
# создаем переменные для удобства и ненагромаждения кода
filter_06 = data_22.release_date.dt.month == 6
filter_07 = data_22.release_date.dt.month == 7
filter_08 = data_22.release_date.dt.month == 8

data_22 = data_22[filter_06 | filter_07 | filter_08] # делаем срез по необходимым месяцам
data_22.imdb_id.value_counts().sum() # считаем кол-во фильмов суммарно для сезона "лето"

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [68]:
answers['23'] = 'Peter Jackson'
#+

In [4]:
data_23 = data.copy()
data_23 = fun_explode(data_23, 'director')
# фильтры для удобства, как в вопросе №22
filter_12 = data_23.release_date.dt.month == 12 # декабрь
filter_01 = data_23.release_date.dt.month == 1 # январь
filter_02 = data_23.release_date.dt.month == 2 # февраль

# реализуем фильтр и посчитаем наиболее часто встречающегося режиссера в эти месяца
data_23 = data_23[filter_12 | filter_01 | filter_02]
data_23.director.value_counts().index[0]

'Peter Jackson'

In [20]:
data['release_month'] = data.release_date.dt.month
data23 = data.copy()
data23 = fun_explode(data23, 'director')
data23 = data[(data.release_month == 1) | (data.release_month == 2) | (data.release_month == 12)]
data23.director.value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [70]:
answers['24'] = 'Four By Two Productions'
#+

In [7]:
data24 = data.copy()
data24.production_companies = data24.production_companies.str.split('|')
data24 = data24.explode('production_companies')

# добавим новый столбец в датафрейм с кол-вом символов в названии для каждого фильма
data24['count_letter'] = data24.original_title.apply(len)
# группируем по компаниям с подсчетом среднего показателя кол-ва символов в названии, 
# сортируем по убыванию и выбираем первый в списке с индексом 0
data24.groupby(['production_companies'])['count_letter'].mean().sort_values(ascending=False).index[0]

#data24 = data24.groupby(['production_companies'])['original_title'].mean().sort_values(ascending=False)
#data24.head(1)

'Four By Two Productions'

In [22]:
data_24 = data.copy()
data_24 = fun_explode(data_24, 'production_companies').reset_index(drop=True)
# добавим новый столбец в датафрейм с кол-вом символов в названии для каждого фильма
data_24['count_letter'] = data_24.original_title.apply(len)
# группируем по компаниям с подсчетом среднего показателя кол-ва символов в названии, 
# сортируем по убыванию и выбираем первый в списке с индексом 0
data_24.groupby(['production_companies'])['count_letter'].mean().sort_values(ascending=False).index[0]

'Four By Two Productions'

In [38]:
data_24 = data.copy()
# добавим новый столбец в датафрейм с кол-вом символов в названии для каждого фильма
data_24['count_letter'] = data_24.original_title.apply(len)
data_24 = fun_explode(data_24, 'production_companies').reset_index(drop=True)
# группируем по компаниям с подсчетом среднего показателя кол-ва символов в названии, 
# сортируем по убыванию и выбираем первый в списке с индексом 0
data_24.groupby(['production_companies'])['count_letter'].mean().sort_values(ascending=False).index[0]

'Four By Two Productions'

In [6]:
data_24 = data.copy()
data_24['title_symbols'] = data_24.original_title.apply(len)
data_24['production_companies'] = data_24.production_companies.str.split('|')
data_24 = data_24.explode('production_companies')
data_24 = data_24.groupby(['production_companies'])['title_symbols'].mean().sort_values(ascending=False)
data_24

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Polsky Films                  46.0
Museum Canada Productions     46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
Global Entertainment Group     2.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Name: title_symbols, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [72]:
answers['25'] = 'Midnight Picture Show'
#+

In [73]:
data_25 = data.copy()
data_25 = fun_explode(data_25, 'production_companies').reset_index(drop=True)
# добавим новый столбец в датафрейм с кол-вом слов в описании для каждого фильма
data_25['len_over'] = data_25.overview.str.split(' ').apply(len)
# группируем по компаниям с подсчетом среднего показателя кол-ва слов в описании, 
# сортируем по убыванию и выбираем первый в списке с индексом 0
data_25.groupby(['production_companies'])['len_over'].mean().sort_values(ascending=False).index[0]

'Midnight Picture Show'

In [16]:
data['low_words'] = data['overview'].str.split().str.len()
data
#data_25 = data[data.low_words >= data.low_words.mean()]
#data_25
#data_25 = data['production_companies'].str.cat(sep='|')
#data_25 = pd.Series(data_25.split('|'))
#data_25 = data_25.value_counts(ascending=False)
#data_25

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,low_words
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,1363528810,26
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,228436354,110
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,185238201,22
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,7.5,2015,1868178225,26
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,7.3,2015,1316249360,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,2000-07-13,6.6,2000,82299717,27
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,2000-10-27,6.4,2000,-8444012,75
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,2000-06-30,4.0,2000,-40865180,58
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...,Diane Keaton,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,Comedy|Drama,Laurence Mark Productions|Columbia Pictures Co...,2000-02-16,5.2,2000,-3962091,24


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [74]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
#+

In [68]:
excerpt = data['vote_average'].value_counts(bins = 100, normalize = True, sort = False)
print(excerpt.index[94])
display(data[(data['vote_average'] > excerpt.index[94].left)].original_title)

(7.812, 7.86]


9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

In [12]:
data_261 = data.copy()
# для столбца vote_average определим число - процентиль, 
# рейтинг фильмов выше такого числа и будет 1% лучших фильмов
perc_va = np.percentile(data_261['vote_average'], q = 99)
display(perc_va)
data_261[data_261['vote_average'] > perc_va].reset_index(drop=True).original_title

7.8

0                                        Inside Out
1                                              Room
2                                      Interstellar
3                           Guardians of the Galaxy
4                                The Imitation Game
5                                         Gone Girl
6                          The Grand Budapest Hotel
7                                         Inception
8                                   The Dark Knight
9                                       The Pianist
10    The Lord of the Rings: The Return of the King
11                          The Wolf of Wall Street
12                                 12 Years a Slave
13                                          Memento
Name: original_title, dtype: object

In [89]:
data_261 = data.copy()
display(data[(data['vote_average'] > np.percentile(data['vote_average'],99))].original_title)

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

In [78]:
df_26 = data.copy()
df_26_perc = np.percentile(df_26['vote_average'], q = 99)
print(df_26_perc)
df_26[df_26['vote_average'] > df_26_perc].reset_index(drop=True).original_title

7.8


0                                        Inside Out
1                                              Room
2                                      Interstellar
3                           Guardians of the Galaxy
4                                The Imitation Game
5                                         Gone Girl
6                          The Grand Budapest Hotel
7                                         Inception
8                                   The Dark Knight
9                                       The Pianist
10    The Lord of the Rings: The Return of the King
11                          The Wolf of Wall Street
12                                 12 Years a Slave
13                                          Memento
Name: original_title, dtype: object

Вариант 2. То же самое, только функция quantile() вместо percentile().  

In [57]:
data_262 = data.copy()
quant_va = np.quantile(data_262['vote_average'], q = 0.99)
print(quant_va)
data_262[data_262['vote_average'] > quant_va].reset_index(drop=True).original_title

7.8


0                                        Inside Out
1                                              Room
2                                      Interstellar
3                           Guardians of the Galaxy
4                                The Imitation Game
5                                         Gone Girl
6                          The Grand Budapest Hotel
7                                         Inception
8                                   The Dark Knight
9                                       The Pianist
10    The Lord of the Rings: The Return of the King
11                          The Wolf of Wall Street
12                                 12 Years a Slave
13                                          Memento
Name: original_title, dtype: object

Вариант 3. Ответ немного другой...

In [82]:
data_26 = data.copy()
count = round(len(data_26) / 100)
print(count,'фильмов входит в 1% от общего рейтинга')
print('Ниже список таких фильмов:')
# сортируем по полю 'vote_average' по убывающей 
# и показываем первые "count" строк - это и есть перечень топ 1% из общего датафрейма
data_26.sort_values(by='vote_average', ascending = False).reset_index(drop=True).head(count)

19 фильмов входит в 1% от общего рейтинга
Ниже список таких фильмов:


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008
1,tt0816692,165000000,621752480,Interstellar,"[Matthew McConaughey, Jessica Chastain, Anne H...",Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,11/5/2014,8.0,2014
2,tt2084970,14000000,233555708,The Imitation Game,"[Benedict Cumberbatch, Keira Knightley, Matthe...",Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,11/14/2014,8.0,2014
3,tt2096673,175000000,853708609,Inside Out,"[Amy Poehler, Phyllis Smith, Richard Kind, Bil...",Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015
4,tt3170832,6000000,35401758,Room,"[Brie Larson, Jacob Tremblay, Joan Allen, Sean...",Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015
5,tt0993846,100000000,392000694,The Wolf of Wall Street,"[Leonardo DiCaprio, Jonah Hill, Margot Robbie,...",Martin Scorsese,EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,Crime|Drama|Comedy,Paramount Pictures|Appian Way|EMJAG Production...,12/25/2013,7.9,2013
6,tt2267998,61000000,369330363,Gone Girl,"[Ben Affleck, Rosamund Pike, Carrie Coon, Neil...",David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,10/1/2014,7.9,2014
7,tt2024544,20000000,187000000,12 Years a Slave,"[Chiwetel Ejiofor, Michael Fassbender, Lupita ...",Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,Plan B Entertainment|Regency Enterprises|River...,10/18/2013,7.9,2013
8,tt2015381,170000000,773312399,Guardians of the Galaxy,"[Chris Pratt, Zoe Saldana, Dave Bautista, Vin ...",James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014
9,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,"[Elijah Wood, Ian McKellen, Viggo Mortensen, L...",Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,WingNut Films|New Line Cinema,12/1/2003,7.9,2003


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [78]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
#+

In [112]:
data_27 = data.copy()
data_27 = data_27.cast.str.split('|') # получаем строки, каждая из которых - список актеров того или иного фильма
cast_comb = []
# пройдемся по data_27 и для каждой строки найдем все возможные пары, добавим их все в список cast_comb
for i in range(0,len(data_27)):
    cast_comb.append(list(combinations(sorted(data_27[i]),2)))
# посчитаем количество встреч той или иной пары и отсортируем по убыванию
Counter(chain(*cast_comb)).most_common(10)

[(('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Emma Watson', 'Rupert Grint'), 8),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Helena Bonham Carter', 'Johnny Depp'), 6),
 (('Paul Walker', 'Vin Diesel'), 5),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5)]

Вариант 2. То же самое, только permutations вместо combinations. Мне кажется, такой вариант предпочтительнее. 
Это наглядно видно на паре ('Rupert Grint', 'Emma Watson').

In [32]:
data_272 = data.copy()
data_272 = data_272.cast.str.split('|')
cast_comb = []
for i in range(0,len(data_272)):
    cast_i = list(permutations(data_272[i],2))
    cast_comb += cast_i
Counter(cast_comb).most_common(10)
len(cast_comb)

NameError: name 'permutations' is not defined

In [33]:
pairs_list = data.cast.apply(lambda x: list(combinations(sorted(x.split('|')), 2))).tolist()
Counter(chain(*pairs_list)).most_common(10)
#Counter(chain(*data.cast.apply(lambda x: list(combinations(sorted(x.split('|')), 2))).tolist())).most_common(10)

NameError: name 'chain' is not defined

In [36]:
df_27 = data.copy()
# Разбиваем актеров по разделителю '|' и делаем в каждой строке списки из комбинаций разных пар, отсортированных 
# т.е. пара (b,a) будет записана как (a,b): 
#df_27.cast = df_27.cast.apply(lambda x: list(combinations(sorted(x.split('|')), 2)))
# Собираем этот столбец в единый плоский список всех пар 
#pairs_lst = df_27.cast.sum()
# С помощью функции Counter считаем сколько раз встретилась та или иная пара и берем 10 наиболее часто встречающихся
#Counter(pairs_lst).most_common(10)

# Или все запишем в одну строку:
Counter(df_27.cast.apply(lambda x: list(combinations(sorted(x.split('|')), 2))).sum()).most_common(10)

[(('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Emma Watson', 'Rupert Grint'), 8),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Helena Bonham Carter', 'Johnny Depp'), 6),
 (('Paul Walker', 'Vin Diesel'), 5),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5)]

In [9]:
import itertools as abcd
#from itertools import combinations
cast_pairs = data.copy()
cast_pairs['cast'] = cast_pairs.cast.str.split('|')
cast_pairs.cast = cast_pairs.cast.apply(lambda x: list(abcd.combinations(sorted(x), 2)))
cast_pairs.explode('cast').cast.mode()

0     (Daniel Radcliffe, Emma Watson)
1    (Daniel Radcliffe, Rupert Grint)
2         (Emma Watson, Rupert Grint)
dtype: object

In [51]:
cast_pairs = data.copy()
cast_pairs['cast'] = cast_pairs.cast.str.split('|')
cast_pairs.cast = cast_pairs.cast.apply(lambda x: list(combinations(sorted(x), 2))) 
cast_pairs.explode('cast').cast.value_counts()

(Daniel Radcliffe, Rupert Grint)       8
(Daniel Radcliffe, Emma Watson)        8
(Emma Watson, Rupert Grint)            8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(James Franco, Rachel McAdams)         1
(Ron Perlman, Ulrich Thomsen)          1
(Jessica Guadix, Ryan Guzman)          1
(Ryan Reynolds, Tim Robbins)           1
(Edgar RamÃ­rez, Mickey Rourke)        1
Name: cast, Length: 17943, dtype: int64

In [72]:
def stringtolist(string):
    result = string.str.split('|')
    return result
from collections import Counter
data = pd.read_csv('movie_bd_v5.csv')
data['cast'] = stringtolist(data['cast'])
actor_pairs = []
for i in range(0, len(data)):
    for item in combinations(data['cast'][i], 2):
        actor_pairs.append(item)
Counter(actor_pairs).most_common(10)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5),
 (('Robert Pattinson', 'Taylor Lautner'), 5)]

# Submission

In [81]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [82]:
# и убедиться что ни чего не пропустил)
len(answers)

27